In [2]:
import numpy as np
import os
dtype = np.float32

import torch
import torchvision
from tqdm import tqdm
import os

from transformers import CLIPVisionModelWithProjection, ViTForImageClassification, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer
import numpy

from huggingface_hub import try_to_load_from_cache, _CACHED_NO_EXIST
from huggingface_hub import scan_cache_dir

import glob
import random
import json
import os

def contains_all_substrings(string, substrings):
    return all(substring in string for substring in substrings)

/home/jgryu/miniconda3/envs/nicc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Weight matrix의 각 Row 나 col을 데이터로 사용

In [3]:
import torch
from transformers import CLIPVisionModelWithProjection, ViTForImageClassification, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer

ckpt_path = '/home/jgryu/Weight_compression/llm-awq/model_cache/models--meta-llama--Meta-Llama-3-8B/snapshots/8cde5ca8380496c9a6cc7ef3a8b46a0372a1d920'
net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)

mean = np.load(f'/home/jgryu/Weight_compression/Wparam_dataset/TFRecord/meta-llama--Meta-Llama-3-8B/mlp/d16/mlp_d16_train_mean.npy')
std = np.load(f'/home/jgryu/Weight_compression/Wparam_dataset/TFRecord/meta-llama--Meta-Llama-3-8B/mlp/d16/mlp_d16_train_std.npy')
mean = torch.from_numpy(mean)
std = torch.from_numpy(std)

state_dict = net.state_dict()

for k, v in state_dict.items():
    print(k, v.shape)

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.20s/it]

model.embed_tokens.weight torch.Size([128256, 4096])
model.layers.0.self_attn.q_proj.weight torch.Size([4096, 4096])
model.layers.0.self_attn.k_proj.weight torch.Size([1024, 4096])
model.layers.0.self_attn.v_proj.weight torch.Size([1024, 4096])
model.layers.0.self_attn.o_proj.weight torch.Size([4096, 4096])
model.layers.0.mlp.gate_proj.weight torch.Size([14336, 4096])
model.layers.0.mlp.up_proj.weight torch.Size([14336, 4096])
model.layers.0.mlp.down_proj.weight torch.Size([4096, 14336])
model.layers.0.input_layernorm.weight torch.Size([4096])
model.layers.0.post_attention_layernorm.weight torch.Size([4096])
model.layers.1.self_attn.q_proj.weight torch.Size([4096, 4096])
model.layers.1.self_attn.k_proj.weight torch.Size([1024, 4096])
model.layers.1.self_attn.v_proj.weight torch.Size([1024, 4096])
model.layers.1.self_attn.o_proj.weight torch.Size([4096, 4096])
model.layers.1.mlp.gate_proj.weight torch.Size([14336, 4096])
model.layers.1.mlp.up_proj.weight torch.Size([14336, 4096])
model.

In [ ]:
size = 4096
model_name = 'models--meta-llama--Meta-Llama-3-8B'
wtype = 'mlp'
chunk = 'row'

datas = []
for k, v in state_dict.items():
    if wtype not in k: continue
    
    if chunk == 'col':
        v = v.T
    v = v.reshape(-1, 4096)
    
    datas.append(v)
    
datas = torch.cat(datas, dim = 0)

indices = torch.randperm(len(datas))
# split_index = int(len(datas) * 0.99)
split_index = int(len(datas) - 1000)
train_indices = indices[:split_index]
val_indices = indices[split_index:]

dataset = {}
dataset['train'] = datas[train_indices]
dataset['val'] = datas[val_indices]
print(dataset['train'].shape, dataset['val'].shape)

dataset_stats = {}
for split in ['train', 'val']:
    data = dataset[split]
    
    # dim=0 (열 단위) 평균과 분산 계산
    mean_dim0 = data.mean(dim=0)
    std_dim0 = data.std(dim=0)
    
    # 전체 데이터에 대한 평균과 분산 계산
    mean_all = data.mean()
    std_all = data.std()
    
    # 계산된 값 저장
    dataset_stats[split] = {
        'mean_channel': mean_dim0,
        'std_channel': std_dim0,
        'mean': mean_all,
        'std': std_all
    }

import os
os.makedirs(f'./dataset_wp_one_row/{model_name}', exist_ok = True)
torch.save(dataset, f'./dataset_wp_one_row/{model_name}/{wtype}_{size}_dataset.pt')         # dataset 저장
torch.save(dataset_stats, f'./dataset_wp_one_row/{model_name}/{wtype}_{size}_dataset_stats.pt')  # dataset_stats 저장

torch.Size([1375256, 4096]) torch.Size([1000, 4096])


In [2]:
import torch
dataset_stats = torch.load('/home/jgryu/Weight_compression/nic_weight_comp_transformer/dataset_wp_one_row/models--meta-llama--Meta-Llama-3-8B/mlp_4096_dataset_stats.pt')

/tmp/ipykernel_256028/1259562839.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset_stats = torch.load('/home/jgryu/Weight_compression/nic_weight_comp_transformer/d

In [3]:
dataset_stats

{'train': {'mean_channel': tensor([ 6.2498e-05,  5.4989e-05, -1.1206e-04,  ...,  1.9405e-04,
           5.4884e-04,  6.7596e-05]),
  'std_channel': tensor([0.0118, 0.0118, 0.0118,  ..., 0.0117, 0.0119, 0.0119]),
  'mean': tensor(-5.6833e-06),
  'std': tensor(0.0118)},
 'val': {'mean_channel': tensor([-4.5342e-04,  3.4744e-04,  3.2038e-04,  ..., -2.7532e-04,
           7.6866e-04, -9.3252e-05]),
  'std_channel': tensor([0.0118, 0.0118, 0.0119,  ..., 0.0117, 0.0116, 0.0123]),
  'mean': tensor(-3.5879e-06),
  'std': tensor(0.0116)}}